In [ ]:
import numpy as np
import multiprocessing as mp
import time

# Parameters
N = 10**6
a = 3.0

# Generate vectors
np.random.seed(42)
x = np.random.randn(N)
y = np.random.randn(N)

# --- Serial Version ---
start_serial = time.time()
d_serial = a * x + y
end_serial = time.time()
serial_time = end_serial - start_serial

# --- Parallel Version ---
def daxpy_chunk(start, end, x, y, a):
    return a * x[start:end] + y[start:end]

# Prepare chunks
num_chunks = mp.cpu_count()
chunk_size = N // num_chunks
indices = [(i * chunk_size, (i + 1) * chunk_size) for i in range(num_chunks)]
indices[-1] = (indices[-1][0], N)  # Handle remainder

# Run in parallel
start_parallel = time.time()
with mp.Pool(processes=num_chunks) as pool:
    results = pool.starmap(daxpy_chunk, [(start, end, x, y, a) for start, end in indices])
d_parallel = np.concatenate(results)
end_parallel = time.time()
parallel_time = end_parallel - start_parallel

# --- Verification ---
is_equal = np.allclose(d_serial, d_parallel)

# --- Output ---
print("DAXPY Result Correct:", is_equal)
print(f"Serial Time   : {serial_time:.6f} seconds")
print(f"Parallel Time : {parallel_time:.6f} seconds")
